In [3]:
# default_exp engineering.web.django

%reload_ext autoreload
%autoreload 2

# Django REST framework
https://www.django-rest-framework.org/



## install

In [ ]:
pip install django
pip install djangorestframework

## Settings
Add 'rest_framework' to INSTALLED_APPS.

    INSTALLED_APPS = [
        ...
        'rest_framework',
    ]

## Quickstart
https://www.django-rest-framework.org/tutorial/quickstart/#quickstart

## Tutorial 1: Serialization
https://www.django-rest-framework.org/tutorial/1-serialization/#tutorial-1-serialization

序列化 (Serialization)将对象的状态信息转换为可以存储或传输的形式的过程。在序列化期间，对象将其当前状态写入到临时或持久性存储区。以后，可以通过从存储区中读取或反序列化对象的状态，重新创建该对象。

所以一个序列化类serializer就对应一个model 类

### 首先创建一个model

In [ ]:
from django.db import models
from pygments.lexers import get_all_lexers
from pygments.styles import get_all_styles

LEXERS = [item for item in get_all_lexers() if item[1]]
LANGUAGE_CHOICES = sorted([(item[1][0], item[0]) for item in LEXERS])
STYLE_CHOICES = sorted((item, item) for item in get_all_styles())


class Snippet(models.Model):
    created = models.DateTimeField(auto_now_add=True)
    title = models.CharField(max_length=100, blank=True, default='')
    code = models.TextField()
    linenos = models.BooleanField(default=False)
    language = models.CharField(choices=LANGUAGE_CHOICES, default='python', max_length=100)
    style = models.CharField(choices=STYLE_CHOICES, default='friendly', max_length=100)

    class Meta:
        ordering = ('created',)

We'll also need to create an initial migration for our snippet model, and sync the database for the first time.

In [ ]:
python manage.py makemigrations snippets
python manage.py migrate

### then创建一个对应的serializer
The first thing we need to get started on our Web API is to provide a way of serializing and deserializing the snippet instances into representations such as json. We can do this by declaring serializers that work very similar to Django's forms. Create a file in the snippets directory named serializers.py and add the following.

In [ ]:
from rest_framework import serializers
from snippets.models import Snippet, LANGUAGE_CHOICES, STYLE_CHOICES


class SnippetSerializer(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    title = serializers.CharField(required=False, allow_blank=True, max_length=100)
    code = serializers.CharField(style={'base_template': 'textarea.html'})
    linenos = serializers.BooleanField(required=False)
    language = serializers.ChoiceField(choices=LANGUAGE_CHOICES, default='python')
    style = serializers.ChoiceField(choices=STYLE_CHOICES, default='friendly')

    def create(self, validated_data):
        """
        Create and return a new `Snippet` instance, given the validated data.
        """
        return Snippet.objects.create(**validated_data)

    def update(self, instance, validated_data):
        """
        Update and return an existing `Snippet` instance, given the validated data.
        """
        instance.title = validated_data.get('title', instance.title)
        instance.code = validated_data.get('code', instance.code)
        instance.linenos = validated_data.get('linenos', instance.linenos)
        instance.language = validated_data.get('language', instance.language)
        instance.style = validated_data.get('style', instance.style)
        instance.save()
        return instance

A serializer class的第一部分定义了要进行序列化/反序列化的字段。 create（）和update（）方法定义在调用serializer.save（）时如何创建或修改完整实例。

A serializer class与Django Form类非常相似，并且在各个字段上包括相似的验证标志，例如required，max_length和default。

### Using ModelSerializers
可以看到，serializer在定义时和model有很多重复的信息，一个简洁的方式就是使用ModelSerializers

It's important to remember that ModelSerializer classes don't do anything particularly magical, they are simply a shortcut for creating serializer classes:

An automatically determined set of fields.

Simple default implementations for the create() and update() methods.

In [ ]:
class SnippetSerializer(serializers.ModelSerializer):
    class Meta:
        model = Snippet
        fields = ('id', 'title', 'code', 'linenos', 'language', 'style')

## Tutorial 3: Class-based Views
https://www.django-rest-framework.org/tutorial/3-class-based-views/#tutorial-3-class-based-views

We can also write our API views using class-based views, rather than function based views. As we'll see this is a powerful pattern that allows us to reuse common functionality, and helps us keep our code DRY(Don't repeat yourself, is a principle of software development aimed at reducing repetition of software patterns,[1] replacing it with abstractions or using data normalization to avoid redundancy. ).

### Rewriting our API using class-based views


In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from django.http import Http404
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status


class SnippetList(APIView):
    """
    List all snippets, or create a new snippet.
    """
    def get(self, request, format=None):
        snippets = Snippet.objects.all()
        serializer = SnippetSerializer(snippets, many=True)
        return Response(serializer.data)

    def post(self, request, format=None):
        serializer = SnippetSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

In [ ]:
class SnippetDetail(APIView):
    """
    Retrieve, update or delete a snippet instance.
    """
    def get_object(self, pk):
        try:
            return Snippet.objects.get(pk=pk)
        except Snippet.DoesNotExist:
            raise Http404

    def get(self, request, pk, format=None):
        snippet = self.get_object(pk)
        serializer = SnippetSerializer(snippet)
        return Response(serializer.data)

    def put(self, request, pk, format=None):
        snippet = self.get_object(pk)
        serializer = SnippetSerializer(snippet, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    def delete(self, request, pk, format=None):
        snippet = self.get_object(pk)
        snippet.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)

We'll also need to refactor our snippets/urls.py slightly now that we're using class-based views.

In [ ]:
from django.urls import path
from rest_framework.urlpatterns import format_suffix_patterns
from snippets import views

urlpatterns = [
    path('snippets/', views.SnippetList.as_view()),
    path('snippets/<int:pk>/', views.SnippetDetail.as_view()),
]

urlpatterns = format_suffix_patterns(urlpatterns)

### Using mixins
使用基于类的视图的最大胜利之一是它使我们能够轻松地组合可重用的行为。

到目前为止，对于我们创建的任何模型支持的API视图，我们一直在使用的create / retrieve / update / delete操作将非常相似。 这些常见行为在REST框架的mixin类中实现。

让我们看一下如何使用mixin类组成视图。 再次是我们的views.py模块。

In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from rest_framework import mixins
from rest_framework import generics

class SnippetList(mixins.ListModelMixin,
                  mixins.CreateModelMixin,
                  generics.GenericAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer

    def get(self, request, *args, **kwargs):
        return self.list(request, *args, **kwargs)

    def post(self, request, *args, **kwargs):
        return self.create(request, *args, **kwargs)

我们将花一点时间来检查一下这里到底发生了什么。 我们正在使用GenericAPIView构建视图，并添加ListModelMixin和CreateModelMixin。

基类提供核心功能，mixin类提供.list（）和.create（）操作。 然后，我们将get和post方法显式绑定到适当的操作。 到目前为止，足够简单的东西。

In [ ]:
class SnippetDetail(mixins.RetrieveModelMixin,
                    mixins.UpdateModelMixin,
                    mixins.DestroyModelMixin,
                    generics.GenericAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer

    def get(self, request, *args, **kwargs):
        return self.retrieve(request, *args, **kwargs)

    def put(self, request, *args, **kwargs):
        return self.update(request, *args, **kwargs)

    def delete(self, request, *args, **kwargs):
        return self.destroy(request, *args, **kwargs)

Pretty similar. Again we're using the GenericAPIView class to provide the core functionality, and adding in mixins to provide the .retrieve(), .update() and .destroy() actions.

### Using generic class-based views
使用mixin类，我们重写了视图，以使用比以前更少的代码，但是我们可以更进一步。 REST框架提供了一组已经混入的通用视图，我们可以使用它们来进一步缩小views.py模块。

In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from rest_framework import generics


class SnippetList(generics.ListCreateAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer


class SnippetDetail(generics.RetrieveUpdateDestroyAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer

哇，太简洁了。我们免费获得了大量的代码，我们的代码看起来就像一个良好、干净、符合惯例的Django。

## Tutorial 6: ViewSets & Routers
https://www.django-rest-framework.org/tutorial/6-viewsets-and-routers/#tutorial-6-viewsets-routers


# drf-yasg
https://drf-yasg.readthedocs.io/en/1.17.1/readme.html

Generate real Swagger/OpenAPI 2.0 specifications from a Django Rest Framework API.

Compatible with

    Django Rest Framework: 3.8, 3.9, 3.10, 3.11
    Django: 1.11, 2.2, 3.0
    Python: 2.7, 3.6, 3.7, 3.8






## The @swagger_auto_schema decorator
https://drf-yasg.readthedocs.io/en/1.17.1/custom_spec.html#the-swagger-auto-schema-decorator

You can use the `@swagger_auto_schema` decorator on view functions to override some properties of the generated Operation. 

For example, in a ViewSet,

In [ ]:
from drf_yasg.utils import swagger_auto_schema

@swagger_auto_schema(operation_description="partial_update description override", 
                     responses={404: 'slug not found'})
def partial_update(self, request, *args, **kwargs):
   """partial_update method docstring"""
   ...

# Django REST Swagger
https://django-rest-swagger.readthedocs.io/en/latest/

Swagger/OpenAPI Documentation Generator for Django REST Framework。这个和drf-yasg干的是一件事情。可惜我选择的是这个，zy选择的是drf-yasg。

# django-filter
https://django-filter.readthedocs.io/en/master/
    
Django-filter是一个通用的，可重用的应用程序，可减轻编写视图代码中一些平凡的工作的负担。 具体来说，它允许用户根据模型的字段过滤查询集，并显示表单以允许他们执行此操作。
## install

In [1]:
# !pip install django-filter
!pip freeze | grep django

django-filter==2.2.0
django-rest-swagger==2.2.0
djangorestframework==3.9.3


In [ ]:
# Then add 'django_filters' to your INSTALLED_APPS.
INSTALLED_APPS = [
    ...
    'django_filters',
]

## Integration with DRF(Django Rest Framework)
通过特定于DRF的FilterSet和 filter backend提供了与Django Rest Framework的集成。 这些可以在rest_framework子包中找到。

In [ ]:
# Your view class will also need to add DjangoFilterBackend to the filter_backends.

from django_filters import rest_framework as filters

class ProductList(generics.ListAPIView):
    queryset = Product.objects.all()
    serializer_class = ProductSerializer
    filter_backends = (filters.DjangoFilterBackend,)
    filterset_fields = ('category', 'in_stock')


If you want to use the django-filter backend by default, add it to the DEFAULT_FILTER_BACKENDS setting.

In [ ]:
# settings.py
INSTALLED_APPS = [
    ...
    'rest_framework',
    'django_filters',
]

REST_FRAMEWORK = {
    'DEFAULT_FILTER_BACKENDS': (
        'django_filters.rest_framework.DjangoFilterBackend',
        ...
    ),
}

# nb_export

In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 00_template.ipynb.
Converted active_learning.ipynb.
Converted algo_dl_keras.ipynb.
Converted algo_dl_loss.ipynb.
Converted algo_dl_optimizers.ipynb.
Converted algo_dl_pytorch.ipynb.
Converted algo_ml_tree_catboost.ipynb.
Converted algo_ml_tree_lgb.ipynb.
Converted algo_rs_match_associated_rules.ipynb.
Converted algo_rs_match_deepmatch.ipynb.
Converted algo_rs_match_matrix.ipynb.
Converted algo_rs_search_vector_faiss.ipynb.
Converted algo_seq_embeding.ipynb.
Converted algo_seq_embeding_glove.ipynb.
Converted algo_seq_features_extraction_text.ipynb.
Converted data-processing-eda.ipynb.
Converted data-processing-tf_data.ipynb.
Converted data_processing_split.ipynb.
Converted datastructure_dict_list_set.ipynb.
Converted datastructure_generator.ipynb.
Converted datastructure_matrix_sparse.ipynb.
Converted engineering-colab-kagglelab.ipynb.
Converted engineering_concurrency.ipynb.
Converted engineering_docker.ipynb.
Converted engineering_gc.ipynb.
Converted

In [7]:
!nbdev_build_docs

No notebooks were modified
converting /Users/luoyonggui/PycharmProjects/nbdevlib/index.ipynb to README.md
